# Analysis of datasets

## Import section

In [1]:
import os
import pickle
from functools import partial

import numpy as np
import pandas as pd
from multiprocess.pool import Pool
from rdkit import Chem
from tqdm import tqdm

from ptgnn.features.chienn.molecule3d import smiles_to_3d_mol

## Load dataset (csv's) (assumes to be downloaded in dev/source by default)

In [2]:
ds_source_path = "dev/src/"

In [3]:
dataset_dict = {}

In [4]:
# rs dataset
with open(os.path.join(ds_source_path, "rs", "raw", 'train.pickle'), 'rb') as f:
    split_df = pickle.load(f)

dataset_dict['rs'] = split_df.rdkit_mol_cistrans_stereo.tolist()

In [5]:
split_df

,ID,SMILES_nostereo,rdkit_mol_cistrans_stereo,RS_label,RS_label_binary
0,BrC1=C[C@@H](c2ccc(-c3ccccc3)cc2)CC(c2nc(-c3cc...,BrC1=CC(c2ccc(-c3ccccc3)cc2)CC(c2nc(-c3ccccc3)...,<rdkit.Chem.rdchem.Mol object at 0x000001A65F9...,S,1
1,BrC1=C[C@@H](c2ccc(-c3ccccc3)cc2)CC(c2nc(-c3cc...,BrC1=CC(c2ccc(-c3ccccc3)cc2)CC(c2nc(-c3ccccc3)...,<rdkit.Chem.rdchem.Mol object at 0x000001A65F9...,S,1
2,BrC1=C[C@@H](c2ccc(-c3ccccc3)cc2)CC(c2nc(-c3cc...,BrC1=CC(c2ccc(-c3ccccc3)cc2)CC(c2nc(-c3ccccc3)...,<rdkit.Chem.rdchem.Mol object at 0x000001A65F9...,S,1
3,BrC1=C[C@@H](c2ccc(-c3ccccc3)cc2)CC(c2nc(-c3cc...,BrC1=CC(c2ccc(-c3ccccc3)cc2)CC(c2nc(-c3ccccc3)...,<rdkit.Chem.rdchem.Mol object at 0x000001A65F9...,S,1
4,BrC1=C[C@@H](c2ccc(-c3ccccc3)cc2)CC(c2nc(-c3cc...,BrC1=CC(c2ccc(-c3ccccc3)cc2)CC(c2nc(-c3ccccc3)...,<rdkit.Chem.rdchem.Mol object at 0x000001A65F9...,S,1
...,...,...,...,...,...
326860,c1nnc([C@H]2CNCCO2)n1C1CC1,c1nnc(C2CNCCO2)n1C1CC1,<rdkit.Chem.rdchem.Mol object at 0x000001A7161...,R,0
326861,c1nnc([C@H]2CNCCO2)n1C1CC1,c1nnc(C2CNCCO2)n1C1CC1,<rdkit.Chem.rdchem.Mol object at 0x000001A7161...,R,0
326862,c1nnc([C@H]2CNCCO2)n1C1CC1,c1nnc(C2CNCCO2)n1C1CC1,<rdkit.Chem.rdchem.Mol object at 0x000001A7161...,R,0
326863,c1nnc([C@H]2CNCCO2)n1C1CC1,c1nnc(C2CNCCO2)n1C1CC1,<rdkit.Chem.rdchem.Mol object at 0x000001A7161...,R,0


In [6]:
# ba dataset
with open(os.path.join(ds_source_path, "bindingaffinity", "raw", 'train.pickle'), 'rb') as f:
    split_df = pickle.load(f)

dataset_dict['ba'] = split_df.rdkit_mol_cistrans_stereo.tolist()

In [7]:
split_df

,ID,SMILES_nostereo,rdkit_mol_cistrans_stereo,score0,score1,score2,top_score,range_scores,mean_score,top_score_enantiomers_range
0,BrC1([C@@H]2CC23CC3)CC1,BrC1(C2CC23CC3)CC1,<rdkit.Chem.rdchem.Mol object at 0x000001A696B...,-4.6,-4.6,-4.6,-4.6,0.0,-4.6,0.4
1,BrC1([C@@H]2CC23CC3)CC1,BrC1(C2CC23CC3)CC1,<rdkit.Chem.rdchem.Mol object at 0x000001A696B...,-4.6,-4.6,-4.6,-4.6,0.0,-4.6,0.4
2,BrC1([C@@H]2CC23CC3)CC1,BrC1(C2CC23CC3)CC1,<rdkit.Chem.rdchem.Mol object at 0x000001A696B...,-4.6,-4.6,-4.6,-4.6,0.0,-4.6,0.4
3,BrC1([C@H]2CC23CC3)CC1,BrC1(C2CC23CC3)CC1,<rdkit.Chem.rdchem.Mol object at 0x000001A696B...,-5.0,-5.0,-5.0,-5.0,0.0,-5.0,0.4
4,BrC1([C@H]2CC23CC3)CC1,BrC1(C2CC23CC3)CC1,<rdkit.Chem.rdchem.Mol object at 0x000001A696B...,-5.0,-5.0,-5.0,-5.0,0.0,-5.0,0.4
...,...,...,...,...,...,...,...,...,...,...
234617,c1snnc1C[C@H]1CCCCN1,c1snnc1CC1CCCCN1,<rdkit.Chem.rdchem.Mol object at 0x000001A7808...,-5.1,-5.1,-5.1,-5.1,0.0,-5.1,0.5
234618,c1snnc1C[C@H]1CCCCN1,c1snnc1CC1CCCCN1,<rdkit.Chem.rdchem.Mol object at 0x000001A7808...,-5.1,-5.1,-5.1,-5.1,0.0,-5.1,0.5
234619,c1snnc1C[C@H]1CCCCN1,c1snnc1CC1CCCCN1,<rdkit.Chem.rdchem.Mol object at 0x000001A7808...,-5.1,-5.1,-5.1,-5.1,0.0,-5.1,0.5
234620,c1snnc1C[C@H]1CCCCN1,c1snnc1CC1CCCCN1,<rdkit.Chem.rdchem.Mol object at 0x000001A7808...,-5.1,-5.1,-5.1,-5.1,0.0,-5.1,0.5


In [8]:
# bace
df = pd.read_csv(os.path.join(ds_source_path, "bace", "raw", 'bace.csv'))

fn = partial(
    smiles_to_3d_mol,
    max_number_of_attempts=100,
    max_number_of_atoms=100
)

with Pool(processes=os.cpu_count()) as p:
    dataset_dict['bace'] = list(p.imap(
        fn,
        tqdm(df.mol.tolist(), position=0)
    ))

100%|██████████| 1513/1513 [00:17<00:00, 87.65it/s] 


In [9]:
df

,mol,CID,Class,Model,pIC50,MW,AlogP,HBA,HBD,RB,...,PEOE6 (PEOE6),PEOE7 (PEOE7),PEOE8 (PEOE8),PEOE9 (PEOE9),PEOE10 (PEOE10),PEOE11 (PEOE11),PEOE12 (PEOE12),PEOE13 (PEOE13),PEOE14 (PEOE14),canvasUID
0,O1CC[C@@H](NC(=O)[C@@H](Cc2cc3cc(ccc3nc2N)-c2c...,BACE_1,1,Train,9.154901,431.56979,4.4014,3,2,5,...,53.205711,78.640335,226.855410,107.434910,37.133846,0.000000,7.980170,0.000000,0.000000,1
1,Fc1cc(cc(F)c1)C[C@H](NC(=O)[C@@H](N1CC[C@](NC(...,BACE_2,1,Train,8.853872,657.81073,2.6412,5,4,16,...,73.817162,47.171600,365.676940,174.076750,34.923889,7.980170,24.148668,0.000000,24.663788,2
2,S1(=O)(=O)N(c2cc(cc3c2n(cc3CC)CC1)C(=O)N[C@H](...,BACE_3,1,Train,8.698970,591.74091,2.5499,4,3,11,...,70.365707,47.941147,192.406520,255.752550,23.654478,0.230159,15.879790,0.000000,24.663788,3
3,S1(=O)(=O)C[C@@H](Cc2cc(O[C@H](COCC)C(F)(F)F)c...,BACE_4,1,Train,8.698970,591.67828,3.1680,4,3,12,...,56.657166,37.954151,194.353040,202.763350,36.498634,0.980913,8.188327,0.000000,26.385181,4
4,S1(=O)(=O)N(c2cc(cc3c2n(cc3CC)CC1)C(=O)N[C@H](...,BACE_5,1,Train,8.698970,629.71283,3.5086,3,3,11,...,78.945702,39.361153,179.712880,220.461300,23.654478,0.230159,15.879790,0.000000,26.100143,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1508,Clc1cc2nc(n(c2cc1)C(CC(=O)NCC1CCOCC1)CC)N,BACE_1543,0,Test,3.000000,364.86969,2.5942,3,2,6,...,37.212799,37.681076,180.226410,95.670128,30.107586,9.368159,7.980170,0.000000,0.000000,1543
1509,Clc1cc2nc(n(c2cc1)C(CC(=O)NCc1ncccc1)CC)N,BACE_1544,0,Test,3.000000,357.83731,2.8229,3,2,6,...,45.792797,47.349350,122.401500,99.877144,30.107586,9.368159,7.980170,0.000000,0.000000,1544
1510,Brc1cc(ccc1)C1CC1C=1N=C(N)N(C)C(=O)C=1,BACE_1545,0,Test,2.953115,320.18451,3.0895,2,1,2,...,47.790600,22.563574,96.290794,58.798935,20.071724,9.368159,0.000000,6.904104,0.000000,1545
1511,O=C1N(C)C(=NC(=C1)C1CC1c1cc(ccc1)-c1ccccc1)N,BACE_1546,0,Test,2.733298,317.38440,3.8595,2,1,3,...,77.219978,9.316234,95.907784,112.609720,20.071724,9.368159,0.000000,6.904104,0.000000,1546


In [10]:
# tox
df = pd.read_csv(os.path.join(ds_source_path, "tox21", "raw", 'tox21.csv.gz'))

with Pool(processes=os.cpu_count()) as p:
    dataset_dict['tox'] = list(p.imap(
        fn,
        tqdm(df.smiles.tolist(), position=0)
    ))

100%|██████████| 7831/7831 [00:35<00:00, 222.87it/s]


In [11]:
df

,NR-AR,NR-AR-LBD,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53,mol_id,smiles
0,0.0,0.0,1.0,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,TOX3021,CCOc1ccc2nc(S(N)(=O)=O)sc2c1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,TOX3020,CCN1C(=O)NC(c2ccccc2)C1=O
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,TOX3024,CC[C@]1(O)CC[C@H]2[C@@H]3CCC4=CCCC[C@@H]4[C@H]...
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,TOX3027,CCCN(CC)C(CC)C(=O)Nc1c(C)cccc1C
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOX20800,CC(O)(P(=O)(O)O)P(=O)(O)O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,TOX2725,CCOc1nc2cccc(C(=O)O)c2n1Cc1ccc(-c2ccccc2-c2nnn...
7827,1.0,1.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,TOX2370,CC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(...
7828,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,TOX2371,C[C@]12CC[C@H]3[C@@H](CCC4=CC(=O)CC[C@@]43C)[C...
7829,1.0,1.0,0.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,TOX2377,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1CC[C...


In [12]:
dataset_dict = {
    key: [
        v
        for v in value
        if v is not None
    ]
    for key, value in dataset_dict.items()
}

## Check for number of stereo-centers

In [13]:
test_mol = dataset_dict['rs'][0]

In [14]:
def stereo_center_count(molecule):
    from rdkit import Chem
    return len(Chem.FindMolChiralCenters(molecule))

In [15]:
results = {}

for key in dataset_dict.keys():
    with Pool(processes=os.cpu_count()) as p:
        results[key] = np.unique(np.array(list(p.imap(
            stereo_center_count,
            tqdm(dataset_dict[key], position=0)
        ))), return_counts=True)

100%|██████████| 7818/7818 [00:02<00:00, 2677.45it/s]


In [16]:
results

{'rs': (array([1]), array([326865], dtype=int64)),
 'ba': (array([1]), array([234622], dtype=int64)),
 'bace': (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
  array([1130,  178,   87,   65,   35,    8,    3,    2,    3,    1,    1],
        dtype=int64)),
 'tox': (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
         17, 18, 19, 20, 21, 23, 24, 25, 26, 30, 31, 35, 38, 40]),
  array([6525,  260,  227,  160,  160,   86,   99,   65,   61,   55,   30,
            9,    6,    4,    9,   12,    4,    3,   10,   11,    5,    3,
            1,    4,    2,    2,    1,    1,    1,    1,    1], dtype=int64))}

## Check for number of double bonds with stereo information

In [17]:
def cis_trans_count(molecule):
    from rdkit import Chem
    return sum([
        (bond.GetBondType() == Chem.rdchem.BondType.DOUBLE) and (
            bond.GetStereo() in [
                Chem.rdchem.BondStereo.STEREOZ,
                Chem.rdchem.BondStereo.STEREOE
            ]
        )
        for bond in molecule.GetBonds()
    ])

In [18]:
results = {}

for key in dataset_dict.keys():
    with Pool(processes=os.cpu_count()) as p:
        results[key] = np.unique(np.array(list(p.imap(
            cis_trans_count,
            tqdm(dataset_dict[key], position=0)
        ))), return_counts=True)

100%|██████████| 7818/7818 [00:03<00:00, 2249.62it/s]


In [19]:
results

{'rs': (array([0, 1, 2, 3, 4, 5, 6, 7]),
  array([295161,  29929,   1218,    281,    216,     30,     10,     20],
        dtype=int64)),
 'ba': (array([0, 1, 2, 3, 4, 5]),
  array([222990,  10362,   1165,     71,     24,     10], dtype=int64)),
 'bace': (array([0, 1, 2]), array([1458,   54,    1], dtype=int64)),
 'tox': (array([ 0,  1,  2,  3,  4,  5,  7,  9, 11]),
  array([7354,  359,   69,   12,   14,    1,    1,    7,    1], dtype=int64))}

In [16]:
def cis_trans_count(molecule):
    from rdkit import Chem
    return sum([
        (bond.GetBondType() == Chem.rdchem.BondType.DOUBLE) and (
            bond.GetStereo() in [
                Chem.rdchem.BondStereo.STEREOCIS,
                Chem.rdchem.BondStereo.STEREOTRANS,
            ]
        )
        for bond in molecule.GetBonds()
    ])

In [17]:
results = {}

for key in dataset_dict.keys():
    with Pool(processes=os.cpu_count()) as p:
        results[key] = np.unique(np.array(list(p.imap(
            cis_trans_count,
            tqdm(dataset_dict[key], position=0)
        ))), return_counts=True)[1]

100%|██████████| 7818/7818 [00:03<00:00, 2286.12it/s]


In [18]:
results

{'rs': array([326865], dtype=int64),
 'ba': array([234622], dtype=int64),
 'bace': array([1513], dtype=int64),
 'tox': array([7818], dtype=int64)}

In [17]:
def cis_trans_count(molecule):
    from rdkit import Chem
    return sum([
        (bond.GetBondType() == Chem.rdchem.BondType.DOUBLE) and (
            bond.GetStereo() in [
                Chem.rdchem.BondStereo.STEREOCIS,
                Chem.rdchem.BondStereo.STEREOTRANS,
                Chem.rdchem.BondStereo.STEREOZ,
                Chem.rdchem.BondStereo.STEREOE
            ]
        )
        for bond in molecule.GetBonds()
    ])

In [18]:
results = {}

for key in dataset_dict.keys():
    with Pool(processes=os.cpu_count()) as p:
        results[key] = np.unique(np.array(list(p.imap(
            cis_trans_count,
            tqdm(dataset_dict[key], position=0)
        ))), return_counts=True)[1]

100%|██████████| 7818/7818 [00:03<00:00, 2274.18it/s]


In [19]:
results

{'rs': array([295161,  29929,   1218,    281,    216,     30,     10,     20],
       dtype=int64),
 'ba': array([222990,  10362,   1165,     71,     24,     10], dtype=int64),
 'bace': array([1458,   54,    1], dtype=int64),
 'tox': array([7354,  359,   69,   12,   14,    1,    1,    7,    1], dtype=int64)}

In [22]:
def cis_trans_count(molecule):
    from rdkit import Chem
    return sum([
        (bond.GetBondType() == Chem.rdchem.BondType.DOUBLE) and
        (
            bond.GetStereo() in [
            Chem.rdchem.BondStereo.STEREOZ,
            Chem.rdchem.BondStereo.STEREOE
            ]
        ) and
        (bond.GetBeginAtom().GetDegree() == 3) and
        (bond.GetEndAtom().GetDegree() == 3)
        for bond in molecule.GetBonds()
    ])

In [23]:
results = {}

for key in dataset_dict.keys():
    with Pool(processes=os.cpu_count()) as p:
        results[key] = np.unique(np.array(list(p.imap(
            cis_trans_count,
            tqdm(dataset_dict[key], position=0)
        ))), return_counts=True)[1]

100%|██████████| 7818/7818 [00:03<00:00, 2115.42it/s]


In [24]:
results

{'rs': array([322273,   4582,     10], dtype=int64),
 'ba': array([234063,    559], dtype=int64),
 'bace': array([1484,   28,    1], dtype=int64),
 'tox': array([7472,  255,   58,    9,   14,    1,    1,    7,    1], dtype=int64)}

## Check number of rings

In [13]:
def n_rings_count(molecule):
    return len(molecule.GetRingInfo().AtomRings())

In [14]:
results = {}

for key in dataset_dict.keys():
    with Pool(processes=os.cpu_count()) as p:
        results[key] = np.unique(np.array(list(p.imap(
            n_rings_count,
            tqdm(dataset_dict[key], position=0)
        ))), return_counts=True)

100%|██████████| 7818/7818 [00:02<00:00, 2914.27it/s]


In [15]:
results

{'rs': (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]),
  array([  9785,  54040, 111602,  80860,  45217,  17607,   6461,    772,
            211,    182,     70,     58], dtype=int64)),
 'ba': (array([0, 1, 2, 3, 4, 5]),
  array([ 17142,  98182, 106348,  12702,    240,      8], dtype=int64)),
 'bace': (array([0, 1, 2, 3, 4, 5, 6, 7]),
  array([  5,  13,  43, 512, 656, 251,  30,   3], dtype=int64)),
 'tox': (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 21, 22, 30]),
  array([1804, 2321, 1627, 1020,  653,  244,   78,   35,   17,   12,    2,
            2,    1,    1,    1], dtype=int64))}